In [1]:
# # If running in a notebook, install missing dependencies with pip before import
# import sys
# import subprocess

# def install_and_import(package, import_name=None):
#     import importlib
#     try:
#         if import_name is None:
#             import_name = package
#         importlib.import_module(import_name)
#     except ImportError:
#         print(f"Installing {package} ...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", package])
#         # Optionally try to import again
#         importlib.invalidate_caches()
#         importlib.import_module(import_name)

# # List of (pip_package, import_name) pairs
# packages = [
#     ("torch", "torch"),
#     ("torchvision", "torchvision"),
#     ("datasets", "datasets"),
#     ("numpy", "numpy"),
# ]

# for pip_name, import_name in packages:
#     install_and_import(pip_name, import_name)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ResNet import ResNet
import matplotlib.pyplot as plt
import numpy as np
import gc
import os
import glob
from datasets import load_dataset
from datasets import Dataset

device = "mps" if torch.backends.mps.is_built() \
    else "cuda" if torch.cuda.is_available() else "cpu"

print(device)

torch.manual_seed(3)

mps


## __Using pre-trained model - CIFAR100__

In [33]:
#Im using model weights from pytorch-cifar-models: https://github.com/chenyaofo/pytorch-cifar-models
#Expected accuracy for ResNET56 on CIFAR100 is 72.63%
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_resnet56", pretrained=True).to(device)


Using cache found in /Users/ronibendom/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


#### Load dataset

In [3]:
# Define standard data transforms for CIFAR100
# CIFAR100 mean and std:
# mean = [0.5071, 0.4867, 0.4408], std = [0.2675, 0.2565, 0.2761]

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    ),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    ),
])


In [ ]:
# Load datasets
train_dataset=torchvision.datasets.CIFAR100(root='./data',train=True,download=True,transform=train_transform)
test_dataset=torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform=test_transform)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
#Verify accuracy of pre-trained model
model.eval()
correct_val = 0
total_val = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        _, predicted = torch.max(output, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()
    val_acc = correct_val / total_val if total_val > 0 else 0.

print(val_acc)

Given the results, extracting learning parameters from their log in https://cdn.jsdelivr.net/gh/chenyaofo/pytorch-cifar-models@logs/logs/cifar100/resnet56/default.log

## __Initial training of ResNet-56 on CIFAR100__

#### NN

In [2]:
model=ResNet(num_classes=100,n=9).to(device)

In [4]:
# Load datasets
train_dataset=torchvision.datasets.CIFAR100(root='./data',train=True,download=True,transform=train_transform)
test_dataset=torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform=test_transform)

#### Define basic params

In [ ]:
criterion = nn.CrossEntropyLoss()

# Training hyperparameters
num_epochs = 200
batch_size = 256
initial_lr = 0.1
momentum = 0.9
weight_decay = 5e-4
eta_min = 0.0

# Optimizer
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=initial_lr,
    momentum=momentum,
    weight_decay=weight_decay,
    nesterov=True
)

# Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs,   # total epochs
    eta_min=eta_min  # min LR
)


In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
# Checkpoint loading functionality
def find_latest_checkpoint(checkpoint_dir='checkpoints'):
    """Find the latest checkpoint file in the directory"""
    if not os.path.exists(checkpoint_dir):
        return None
    
    checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'resnet_epoch_*.pth'))
    if not checkpoint_files:
        return None
    
    # Sort by epoch number (extract epoch number from filename)
    def extract_epoch_number(filename):
        # Extract epoch number from filename like 'resnet_epoch_50.pth'
        basename = os.path.basename(filename)
        epoch_str = basename.split('_epoch_')[1].split('.pth')[0]
        return int(epoch_str)
    
    latest_checkpoint = max(checkpoint_files, key=extract_epoch_number)
    return latest_checkpoint

def load_checkpoint(model, optimizer, scheduler, checkpoint_path):
    """Load checkpoint and return epoch number and metrics"""
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint not found: {checkpoint_path}")
        return 0, [], [], []
    
    print(f"Loading checkpoint from: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Load model state
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Load optimizer state
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Load scheduler state
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    # Get epoch number
    start_epoch = checkpoint['epoch']
    
    print(f"Resuming from epoch {start_epoch}")
    print(f"Previous metrics - Loss: {checkpoint['train_loss']:.4f}, "
          f"Train Acc: {checkpoint['train_acc']:.4f}, Val Acc: {checkpoint['val_acc']:.4f}")
    
    return start_epoch, checkpoint.get('train_losses', []), \
           checkpoint.get('train_accuracies', []), checkpoint.get('val_accuracies', [])


In [8]:
# Enhanced training loop with checkpoint resuming
def train_with_resume(model, optimizer, scheduler, criterion, train_loader, test_loader, 
                     num_epochs, device, resume_from_checkpoint=True):
    """
    Training loop that can resume from the latest checkpoint
    """
    # Initialize metrics lists
    train_losses = []
    train_accuracies = []
    val_accuracies = []
    
    # Create directory for saving weights
    os.makedirs('checkpoints', exist_ok=True)
    
    # Try to resume from checkpoint if requested
    start_epoch = 0
    if resume_from_checkpoint:
        latest_checkpoint = find_latest_checkpoint()
        if latest_checkpoint:
            start_epoch, train_losses, train_accuracies, val_accuracies = load_checkpoint(
                model, optimizer, scheduler, latest_checkpoint
            )
            print(f"Resuming training from epoch {start_epoch + 1}")
        else:
            print("No checkpoint found, starting from epoch 1")
    else:
        print("Starting fresh training from epoch 1")
    
    # Training loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        avg_loss = total_loss / len(train_loader)
        train_acc = correct / total if total > 0 else 0.

        del images, labels, output, loss

        model.eval()
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                _, predicted = torch.max(output, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
            val_acc = correct_val / total_val if total_val > 0 else 0.
        
        del images, labels, output

        train_losses.append(avg_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        scheduler.step()

        # Save weights and print every 10th epoch
        if (epoch + 1) % 10 == 0:
            checkpoint_path = f'checkpoints/resnet_epoch_{epoch+1}.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'train_loss': avg_loss,
                'train_acc': train_acc,
                'val_acc': val_acc,
                'train_losses': train_losses,
                'train_accuracies': train_accuracies,
                'val_accuracies': val_accuracies,
            }, checkpoint_path)
            
            print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")
            print(f"Checkpoint saved to {checkpoint_path}")
        
        # Clear memory between epochs
        if device == "cuda":
            torch.cuda.empty_cache()
        elif device == "mps":
            torch.mps.empty_cache()
        # For CPU, we only use gc.collect() which is called below
        gc.collect()
    
    return train_losses, train_accuracies, val_accuracies


In [9]:
# Usage example: Start training with automatic checkpoint resuming
# This will automatically find and load the latest checkpoint if available
train_losses, train_accuracies, val_accuracies = train_with_resume(
    model=model,
    optimizer=optimizer, 
    scheduler=scheduler,
    criterion=criterion,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=num_epochs,
    device=device,
    resume_from_checkpoint=False  # Set to False to start fresh
)


Starting fresh training from epoch 1
Epoch [10/200] Loss: 1.7100 Train Acc: 0.5211 Val Acc: 0.4266
Checkpoint saved to checkpoints/resnet_epoch_10.pth
Epoch [20/200] Loss: 1.2976 Train Acc: 0.6277 Val Acc: 0.4898
Checkpoint saved to checkpoints/resnet_epoch_20.pth
Epoch [30/200] Loss: 1.1403 Train Acc: 0.6664 Val Acc: 0.4885
Checkpoint saved to checkpoints/resnet_epoch_30.pth
Epoch [40/200] Loss: 1.0400 Train Acc: 0.6921 Val Acc: 0.4907
Checkpoint saved to checkpoints/resnet_epoch_40.pth
Epoch [50/200] Loss: 0.9663 Train Acc: 0.7107 Val Acc: 0.5117
Checkpoint saved to checkpoints/resnet_epoch_50.pth
Epoch [60/200] Loss: 0.9090 Train Acc: 0.7259 Val Acc: 0.5390
Checkpoint saved to checkpoints/resnet_epoch_60.pth
Epoch [70/200] Loss: 0.8543 Train Acc: 0.7405 Val Acc: 0.5392
Checkpoint saved to checkpoints/resnet_epoch_70.pth
Epoch [80/200] Loss: 0.7946 Train Acc: 0.7564 Val Acc: 0.5411
Checkpoint saved to checkpoints/resnet_epoch_80.pth
Epoch [90/200] Loss: 0.7241 Train Acc: 0.7785 Val A

In [10]:
torch.manual_seed(1)

In [13]:
# Training hyperparameters
num_epochs = 300
batch_size = 128
initial_lr = 0.1
momentum = 0.9
weight_decay = 5e-4
eta_min = 0.0
T_max = 280

# Optimizer
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=initial_lr,
    momentum=momentum,
    weight_decay=weight_decay,
    nesterov=True
)

# Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=T_max,   # total epochs
    eta_min=eta_min  # min LR
)


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [15]:
# Usage example: Start training with automatic checkpoint resuming
# This will automatically find and load the latest checkpoint if available
train_losses, train_accuracies, val_accuracies = train_with_resume(
    model=model,
    optimizer=optimizer, 
    scheduler=scheduler,
    criterion=criterion,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=num_epochs,
    device=device,
    resume_from_checkpoint=False  # Set to False to start fresh
)


Starting fresh training from epoch 1
Epoch [10/300] Loss: 1.7600 Train Acc: 0.5098 Val Acc: 0.4057
Checkpoint saved to checkpoints/resnet_epoch_10.pth
Epoch [20/300] Loss: 1.5222 Train Acc: 0.5685 Val Acc: 0.4571
Checkpoint saved to checkpoints/resnet_epoch_20.pth
Epoch [30/300] Loss: 1.4403 Train Acc: 0.5900 Val Acc: 0.4880
Checkpoint saved to checkpoints/resnet_epoch_30.pth
Epoch [40/300] Loss: 1.3907 Train Acc: 0.6031 Val Acc: 0.4802
Checkpoint saved to checkpoints/resnet_epoch_40.pth
Epoch [50/300] Loss: 1.3491 Train Acc: 0.6114 Val Acc: 0.5057
Checkpoint saved to checkpoints/resnet_epoch_50.pth
Epoch [60/300] Loss: 1.3098 Train Acc: 0.6227 Val Acc: 0.4951
Checkpoint saved to checkpoints/resnet_epoch_60.pth
Epoch [70/300] Loss: 1.2753 Train Acc: 0.6317 Val Acc: 0.5043
Checkpoint saved to checkpoints/resnet_epoch_70.pth
Epoch [80/300] Loss: 1.2481 Train Acc: 0.6396 Val Acc: 0.5123
Checkpoint saved to checkpoints/resnet_epoch_80.pth
Epoch [90/300] Loss: 1.2154 Train Acc: 0.6491 Val A

In [11]:
# def load_training_data_from_checkpoints(checkpoint_dir):
#     """
#     Load training data from all checkpoints in a directory
#     Returns: epochs, train_losses, val_accuracies
#     """
#     if not os.path.exists(checkpoint_dir):
#         print(f"Directory {checkpoint_dir} does not exist")
#         return [], [], []
    
#     # Get all checkpoint files
#     checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'resnet_epoch_*.pth'))
#     if not checkpoint_files:
#         print(f"No checkpoint files found in {checkpoint_dir}")
#         return [], [], []
    
#     # Sort by epoch number
#     def extract_epoch_number(filename):
#         basename = os.path.basename(filename)
#         epoch_str = basename.split('_epoch_')[1].split('.pth')[0]
#         return int(epoch_str)
    
#     checkpoint_files.sort(key=extract_epoch_number)
    
#     epochs = []
#     train_losses = []
#     val_accuracies = []
    
#     for checkpoint_file in checkpoint_files:
#         try:
#             checkpoint = torch.load(checkpoint_file, map_location='cpu')
#             epoch = checkpoint['epoch']
#             train_loss = checkpoint['train_loss']
#             val_acc = checkpoint['val_acc']
            
#             epochs.append(epoch)
#             train_losses.append(train_loss)
#             val_accuracies.append(val_acc)
            
#         except Exception as e:
#             print(f"Error loading {checkpoint_file}: {e}")
#             continue
    
#     return epochs, train_losses, val_accuracies

# def plot_training_curves(checkpoint_dirs, save_plots=True):
#     """
#     Plot training curves for multiple checkpoint directories
#     """
#     plt.figure(figsize=(15, 5))
    
#     # Plot 1: Training Loss
#     plt.subplot(1, 2, 1)
#     for checkpoint_dir in checkpoint_dirs:
#         epochs, train_losses, val_accuracies = load_training_data_from_checkpoints(checkpoint_dir)
#         if epochs:
#             plt.plot(epochs, train_losses, 'o-', label=f'{checkpoint_dir} (Train Loss)', linewidth=2, markersize=4)
    
#     plt.xlabel('Epoch')
#     plt.ylabel('Training Loss')
#     plt.title('Training Loss vs Epoch')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.yscale('log')  # Log scale for better visualization
    
#     # Plot 2: Validation Accuracy
#     plt.subplot(1, 2, 2)
#     for checkpoint_dir in checkpoint_dirs:
#         epochs, train_losses, val_accuracies = load_training_data_from_checkpoints(checkpoint_dir)
#         if epochs:
#             plt.plot(epochs, val_accuracies, 's-', label=f'{checkpoint_dir} (Val Acc)', linewidth=2, markersize=4)
    
#     plt.xlabel('Epoch')
#     plt.ylabel('Validation Accuracy')
#     plt.title('Validation Accuracy vs Epoch')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
    
#     plt.tight_layout()
    
#     if save_plots:
#         plt.savefig('training_curves_comparison.png', dpi=300, bbox_inches='tight')
#         print("Plot saved as 'training_curves_comparison.png'")
    
#     plt.show()

# # Load and plot data from all checkpoint directories
# checkpoint_directories = [
#     'checkpoints',
#     'checkpoints_weight_decay', 
#     'checkpoints_wrong_scheduler'
# ]

# print("Loading training data from all checkpoint directories...")
# plot_training_curves(checkpoint_directories)


In [12]:
# import plotly.graph_objects as go
# import plotly.express as px
# import os
# import glob
# import torch

# def load_training_data_from_checkpoints_plotly(checkpoint_dir):
#     """
#     Load training data from all checkpoints in a directory for Plotly
#     Returns: epochs, train_losses, val_accuracies
#     """
#     if not os.path.exists(checkpoint_dir):
#         print(f"Directory {checkpoint_dir} does not exist")
#         return [], [], []
    
#     # Get all checkpoint files
#     checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'resnet_epoch_*.pth'))
#     if not checkpoint_files:
#         print(f"No checkpoint files found in {checkpoint_dir}")
#         return [], [], []
    
#     # Sort by epoch number
#     def extract_epoch_number(filename):
#         basename = os.path.basename(filename)
#         epoch_str = basename.split('_epoch_')[1].split('.pth')[0]
#         return int(epoch_str)
    
#     checkpoint_files.sort(key=extract_epoch_number)
    
#     epochs = []
#     train_losses = []
#     val_accuracies = []
    
#     for checkpoint_file in checkpoint_files:
#         try:
#             checkpoint = torch.load(checkpoint_file, map_location='cpu')
#             epoch = checkpoint['epoch']
#             train_loss = checkpoint['train_loss']
#             val_acc = checkpoint['val_acc']
            
#             epochs.append(epoch)
#             train_losses.append(train_loss)
#             val_accuracies.append(val_acc)
            
#         except Exception as e:
#             print(f"Error loading {checkpoint_file}: {e}")
#             continue
    
#     return epochs, train_losses, val_accuracies

# def create_single_plotly_graph(checkpoint_dirs):
#     """
#     Create a single interactive Plotly plot with both training loss and validation accuracy
#     """
#     fig = go.Figure()
    
#     # Color palette for different experiments
#     colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    
#     for i, checkpoint_dir in enumerate(checkpoint_dirs):
#         epochs, train_losses, val_accuracies = load_training_data_from_checkpoints_plotly(checkpoint_dir)
        
#         if epochs:
#             # Clean up directory name for display
#             display_name = checkpoint_dir.replace('checkpoints', '').replace('_', ' ').strip()
#             if not display_name:
#                 display_name = 'Original'
#             else:
#                 display_name = display_name.title()
            
#             # Add training loss trace
#             fig.add_trace(
#                 go.Scatter(
#                     x=epochs, 
#                     y=train_losses,
#                     mode='lines+markers',
#                     name=f'{display_name} - Train Loss',
#                     line=dict(color=colors[i % len(colors)], width=3),
#                     marker=dict(size=6, symbol='circle'),
#                     yaxis='y',
#                     hovertemplate='<b>%{fullData.name}</b><br>' +
#                                 'Epoch: %{x}<br>' +
#                                 'Loss: %{y:.4f}<br>' +
#                                 '<extra></extra>'
#                 )
#             )
            
#             # Add validation accuracy trace
#             fig.add_trace(
#                 go.Scatter(
#                     x=epochs, 
#                     y=val_accuracies,
#                     mode='lines+markers',
#                     name=f'{display_name} - Val Acc',
#                     line=dict(color=colors[i % len(colors)], width=3, dash='dash'),
#                     marker=dict(size=6, symbol='square'),
#                     yaxis='y2',
#                     hovertemplate='<b>%{fullData.name}</b><br>' +
#                                 'Epoch: %{x}<br>' +
#                                 'Accuracy: %{y:.4f}<br>' +
#                                 '<extra></extra>'
#                 )
#             )
    
#     # Update layout with dual y-axes
#     fig.update_layout(
#         title={
#             'text': "Training Progress: Loss and Accuracy Comparison",
#             'x': 0.5,
#             'xanchor': 'center',
#             'font': {'size': 20}
#         },
#         xaxis=dict(title="Epoch"),
#         yaxis=dict(
#             title="Training Loss",
#             type="log",
#             side="left",
#             tickfont=dict(color='#1f77b4')
#         ),
#         yaxis2=dict(
#             title="Validation Accuracy",
#             side="right",
#             overlaying="y",
#             range=[0, 1],
#             tickfont=dict(color='#ff7f0e')
#         ),
#         height=600,
#         width=1000,
#         showlegend=True,
#         template="plotly_white",
#         font=dict(size=12),
#         hovermode='x unified'
#     )
    
#     # Add grid
#     fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
#     fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    
#     return fig

# # Create the single interactive plot
# checkpoint_directories = [
#     'checkpoints',
#     'checkpoints_weight_decay', 
#     'checkpoints_wrong_scheduler'
# ]

# print("Creating single interactive Plotly plot for all checkpoint directories...")
# fig = create_single_plotly_graph(checkpoint_directories)
# fig.show()

# # Save as HTML for sharing
# fig.write_html("training_curves_single.html")
# print("Single interactive plot saved as 'training_curves_single.html'")


In [14]:
# Dataset.cleanup_cache_files
ds = load_dataset("hirundo-io/Noisy-CIFAR-100")

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['png', '__key__', '__url__'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['png', '__key__', '__url__'],
        num_rows: 10000
    })
})

In [17]:
ds['train']

Dataset({
    features: ['png', '__key__', '__url__'],
    num_rows: 50000
})

In [18]:
train_loader = DataLoader(ds['train'], batch_size=256)

In [21]:
train_loader
train_loader.dataset[0]
train_loader.dataset[0]['png']
train_loader.dataset[0]['__key__']
# train_loader.dataset[0]['label']


'train/hamster/04330'

In [17]:
train_loader.dataset[0]['label']


TypeError: tuple indices must be integers or slices, not str

In [16]:
train_loader.dataset[0]

(tensor([[[-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
          [-1.8957, -1.8957,  0.7724,  ...,  1.8426,  1.8426,  1.8426],
          ...,
          [-1.8957, -1.8957, -0.5030,  ...,  0.3913,  0.7285,  0.7724],
          [-1.8957, -1.8957, -0.7229,  ...,  0.4059,  0.4646,  0.4206],
          [-1.8957, -1.8957, -0.7815,  ...,  0.1567,  0.0687,  0.1567]],
 
         [[-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
          [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
          [-1.8975, -1.8975,  1.0686,  ...,  2.0012,  2.0012,  2.0012],
          ...,
          [-1.8975, -1.8975, -0.5979,  ...,  0.9921,  1.4049,  1.4661],
          [-1.8975, -1.8975, -0.7508,  ...,  1.0074,  1.1603,  1.1450],
          [-1.8975, -1.8975, -0.7202,  ...,  0.7781,  0.7475,  0.8392]],
 
         [[-1.5965, -1.5965, -1.5965,  ..., -1.5965, -1.5965, -1.5965],
          [-1.5965, -1.5965,